# Demystifying ICP Purchasing Power Parity (PPP) Calculations

####  Authors: William Vigil-Oliver and Shriya Chauhan

---
Contents
- [Overview](#Overview) 
- [Load required Python libraries](#Libraries)  
- [Load input data](#InputData) 
- [Calculate basic heading PPPs](#BHPPP)  
- [Calculate above-basic heading PPPs](#aBHPPP)  
---

<a id="Overview"></a>

## Overview
This notebook provides the accompanying code for the World Bank blog "Demystifying ICP Purchasing Power Parity (PPP) Calculations". Its purpose is to lay out the calculation steps and showcase the implementation of the main formulas needed to estimate ICP purchasing power parities (PPPs) at the regional level. The blog post is publicly available [here](http://worldbank.org/xxxxx). 

*Note*: Because the target audience may include users unfamiliar with Python or programming in general, we opted to show each calculation step as explicitly as possible, at the cost of having more modularized and computationally efficient code.

<a id="Libraries"></a>

## Load required Python libraries
The code will require loading the following well-known Python libaries: `pandas`, `numpy` and `statsmodels` 

In [ ]:
## Load libaries 
import pandas as pd
import numpy as np 
import statsmodels.api as sm

<a id="InputData"></a>

## Load input data

Let us start by loading input dataset containing mock average price data and other relevant country-level information. 

In [ ]:
#Load price data
data="price_data.csv"
prices=pd.read_csv(data) 
prices # Show full dataset 

This mock dataset contains four countries ('country') and three basic headings ('bh'): garment, rice and pork. Each basic heading contains different varieties of garment, rice and pork. These varieties are contained in the column 'item'. Finally, an average price in local currency unit is reported for each item ('price') and information on the relative importance of each item in a country’s consumption at the basic heading level is contained in the importance column ('imp'). The ICP Technical Advisory Group (TAG) recommended that a weight of '3' be attached to items identified as 'important' and a weight of '1' to items deemed unimportant.

Note that the full ICP classification consists of 155 basic headings.Also,notice that the number of item per basic headings varies from three (in the case of garments) to two (rice and pork). This should not be a problem and is common ocurrence in the ICP as basic headings vary in terms of the number of items they contain. Finally, notice that not all countries report prices for all items. Again, this a common situation in the actual ICP as not all items can be found or are priced in all countries.

<a id="BHPPP"></a>

## Calculate basic heading PPPs

At the first stage, PPPs are first estimated for more than one hundred groups containing similar items. These groups are called 'basic headings' in the ICP literature. The basic heading is also the level of aggregation for which national accounts expenditures are requested to be provided by ICP-participating countries. The end-result of this first calculation stage are a set of several PPPs per country, one PPP for each basic heading per country.

The estimation procedure involves averaging price relatives for individual items from different countries within each basic heading to obtain basic heading level PPPs. This is done via a regression method known as the weighted country product dummy (CPD-W). 

The CPD-W is carried out within each basic heading by regressing the logarithm of the observed country item prices on item dummies (one for each item) and country dummies (one for each country other than the numeraire). The CPD-W method also incorporates information on the relative importance of each item in a country’s consumption. Information about the relative importance of each item is reported by each ICP participating country. 

###  Select the numeraire
This refers to the currrency against which all the estimated PPP values will be compared. In the case of the global PPP results, the numeraire would be the US dollar. In this case, we select the currency of 'country2' as the numeraire.

In [ ]:
## Select the reference or numeraire currency
numeraire = 'country2' 

###  Prep the inputs to run the CPD-W

In [ ]:
## Prep
## Drop country-item observations without a price
prices = prices[prices['price'].notnull()]

## Dataframe with country prices
d_country=pd.get_dummies(prices['country'])

## Prepare design matrix for CPD-W
d_country=pd.get_dummies(prices['country'])
d_country.drop(numeraire, axis=1, inplace=True) #drop numeraire
d_country = d_country.add_prefix('c_') #add prefix to countries
d_item=pd.get_dummies(prices['item'],drop_first=False) #include all item dummies
d_item = d_item.add_prefix('i_') #add prefix to items
prices=pd.concat([prices,d_country,d_item],axis=1) # Concatenate the new cols

## Create empty arrays to store results
l_coef= [] # to store exp(beta_hats)
l_bh= [] # to store bh labels


###  Run the CPD-W on each basic heading and store results

In [ ]:
for bh in prices.bh.unique():
    tempdf=prices[prices.bh == bh] 
    X=tempdf.loc[:, [x for x in tempdf.columns if x.startswith(('c_', 'i_'))]]
    y = np.log(tempdf['price']) 
    wts=tempdf['imp']

    wts_cpd=sm.WLS(y, X,weights=wts)
    res=wts_cpd.fit()
    res_eparams=np.exp(res.params)
    
    print("\n","Basic Heading:", bh, "\n")
    print('Exponentiated Parameters:',"\n",
          res_eparams)
    
    l_coef.append(res_eparams)
    l_bh.append(bh)

coef = np.array(l_coef, dtype=float)
coef = np.round(coef,4) # round to 4 decimals
cols = list(X) #store column heads of X as a list
coef[coef == 1] = np.nan #%% replace PPPs that were exp(0)=1 with 'np.nan'

The results above show the estimated coefficients from the CPD-W method for each of the three basic headings. Of particular interest are the estimated coefficients on the country dummies (denoted by the prefix 'c_') as they are the natural log of the estimated country PPP for the basic heading in question. Note that the estimated coefficients have already been exponentiated.

###  Gather and display the estimated basic heading PPPs 

In [ ]:
#Create dataframe of PPP results from numpy arrays
#dimension = "# BHs" x "# coef"
df_bhppp=pd.DataFrame(data = coef, index = l_bh, columns = cols)
c_numeraire=f"c_{numeraire}"
df_bhppp.insert(0, c_numeraire, 1.000) #insert column of 1s for numeraire

df_bhppp=df_bhppp.loc[:, [x for x in df_bhppp.columns if x.startswith(('c_'))]] #subsetting to store only country level PPPs
df_bhppp.columns = df_bhppp.columns.str.replace('^c_', '') 

#Column sorting function
def sorting(first_col, df):
    columns = df.columns.tolist()
    columns.remove(first_col)
    columns.insert(0,first_col)
    return df.reindex(columns, axis=1)

#Sort cols with numeraire as col1
sorting(numeraire,df_bhppp)

print("\n","Basic Heading PPPs","\n")
print(df_bhppp, "\n")

<a id="aBHPPP"></a>

## Calculate above-basic heading PPPs

At the second stage of calculation, PPPs estimated at the basic heading level are aggregated using national accounts expenditures in local currency for each country as weights.

The aggregation method involves constructing bilateral PPPs for each pair of countries, using basic heading level national accounts expenditures as weights from each country in turn. First, a Laspeyres-type bilateral PPP is calculated between each pair of countries, then a Paasche-type bilateral PPP and finally a Fisher-type bilateral PPP. As a last step, the matrix of Fisher-type bilateral PPPs is made transitive using the Gini-Éltető-Köves-Szulc (GEKS) approach to arrive at a final vector of above basic headings PPPs, containing one PPP for each country relative to the numeraire.

###  Load and display basic heading expenditure values
As a first step let us load the basic heading level national accounts expenditure values in local currency unit for each country.

In [ ]:
#Load basic heading expenditure values
#Should contain bh and countries with prefix c
code="bhdata_exp.csv"
df_bh=pd.read_csv(code,index_col="icp_bh")

In [ ]:
#Sort cols with numeraire as col1
def sorting(first_col, df):
    columns = df.columns.tolist()
    columns.remove(first_col)
    columns.insert(0,first_col)
    return df.reindex(columns, axis=1)

df_bhexp=sorting(c_numeraire,df_bh)

print("\n","Basic Heading Expenditure Values in Local Currency Units","\n")
print(df_bhexp, "\n")

###  Check the basic heading PPP and basic heading expenditure matrices
Before proceeding, it is important to check that both the basic heading PPP and basic heading expenditure matrices have the same dimensions. It is also important to check that the matrix of basic heading PPPs is complete. If the dimensions of the two matrices do not match or the basic heading PPP matrix is incomplete then aggregation at higher aggregate levels is not possible using the formulas employed by the ICP. 

In [ ]:
df_bhexp.columns = df_bhexp.columns.str.replace('^c_', '') 

print("Dimensions of Matrices (no. of headings x no. of countries):","\n")
print("BH Purchasing Power Parities (PPPs)  = ",df_bhppp.shape)
print("BH Nominal Expenditures in LCUs      = ", df_bhexp.shape)

Both matrices have the same dimensions and basic heading PPP matrix is complete, so let us proceed.

###  Calculate bilateral PPPs (Laspeyres-, Paasche-, and Fisher-type)


Calculate the Laspeyres-type bilateral PPPs

In [ ]:
#Calculate Laspeyres bilateral ppps 
shape = (len(df_bhexp.columns),len(df_bhexp.columns))
lp = np.zeros(shape) #square matrix: country x country
nrow= len(lp)  # gets the number of rows
ncol = len(lp[0]) #get the number of cols

for row in range(nrow):
    for col in range(ncol):
        #weighted means by looping over df rows
        lp[row][col]= np.average((df_bhppp.iloc[:,row]/df_bhppp.iloc[:,col]),weights=df_bhexp.iloc[:,col])

lp_ppp = lp
lp_ppp=pd.DataFrame(data = lp_ppp, index = df_bhexp.columns, columns = df_bhexp.columns)
lp_ppp = round(lp_ppp,3)

Square ('country x country') matrix of Laspeyres-type (bilateral) PPPs

In [ ]:
print("\n", "Laspeyres-type bilateral PPPs:","\n")
print(lp_ppp, "\n")

Derive the Paasche-type bilateral PPPs by taking the reciprocal of the transpose of the Laspeyres-type bilateral PPP 

In [ ]:
#Calculate Paasche bilateral ppps 
pa = np.transpose(np.reciprocal(lp))
pa_ppp=pd.DataFrame(data = pa, index = df_bhexp.columns, columns = df_bhexp.columns)
pa_ppp = round(pa_ppp,3)

Square ('country x country') matrix of Paasche-type (bilateral) PPPs

In [ ]:
print("\n", "Paasche-type bilateral PPPs:","\n")
print(pa_ppp, "\n")

Derive the Fisher-type bilateral PPPs by taking the geometric mean of the Laspeyres-type 
and Paasche-type bilateral PPPs for the aggregate

In [ ]:
#Create geomean function
def nangmean(arr, axis=None):
    arr = np.asarray(arr)
    inverse_valids = 1. / np.sum(~np.isnan(arr), axis=axis)  # could be a problem for all-nan-axis
    rhs = inverse_valids * np.nansum(np.log(arr), axis=axis)
    return np.exp(rhs)

#Calculate Fisher bilateral ppps 
fi = np.zeros(shape)
nrow=len(fi)
ncol=len(fi[0])

for row in range(nrow):
    for col in range(ncol):
        fi[row][col]= nangmean([lp[row][col],pa[row][col]])

fi_ppp=pd.DataFrame(data = fi, index = df_bhexp.columns, columns = df_bhexp.columns)
fi_ppp = round(fi_ppp,3)

Square ('country x country') matrix of bilateral (Fisher-type) PPPs

In [ ]:
print("Fisher-type bilateral PPPs:","\n")
print(fi_ppp, "\n")

###  Calculate GEKS PPPs

Finally, let us calculate the GEKS PPPs between each country and the numeraire. The GEKS PPPs for country with respect to the numeraire is given by the geometric mean of the Fisher-type PPP relatives between each pair of countries and the remaining countries. The result is not a matrix of bilateral PPPs but, instead, a vector of PPPs (one PPP per country). 

This vector of PPPs is referred to as 'multilateral PPPs' because the GEKS procedure considers the relative prices between all of the countries as a group. The GEKS is needed to make the Fisher-type bilateral PPPs transitive and base country–invariant. Transitivity simply means that the PPP between any two countries should be the same whether it is computed directly or indirectly through a third country (like with market exchange rates). Base country–invariant means that the PPPs between any two countries should be the same regardless of the choice of base country.

In [ ]:
#Calculate GEKS multilateral ppps 
##reqs the earlier nangmean function 
geks = np.zeros(shape)  # zero 'country x country' matrix
nrow=len(geks)  # gets the number of rows
ncol=len(geks[0])

for row in range(nrow):
    for col in range(ncol):
        geks[row][col]= nangmean(fi[row]/fi[col])     

geks_vec = np.zeros(shape=(1,len(df_bhexp.columns))) # as we need a vector of ppps, not a matrix
j=len(geks_vec[0])
for col in range(j):#..one PPP per country, or col of bhexp df
    geks_vec[:,col]=nangmean(geks[col,0]/geks[0,0]) #geomean over each row, w/ each col rebased to country in col1    

geks_ppp = np.array(geks_vec)
geks_ppp = np.round(geks_ppp,4) # round to 4 decimals

Display the estimated vector of GEKS PPPs, containing one above-basic heading multilateral PPP per country.

In [ ]:
geks_ppp = pd.DataFrame(geks_ppp)
geks_ppp.columns = df_bhexp.columns
geks_ppp = round(geks_ppp,3)

print("\n","GEKS Multilateral PPPs:","\n")
print(geks_ppp.to_string(index=False), "\n")